# Sparkify Churn

<a href="#measureing-churn">Measuring Churn</a>

<a href="#features-engineering">Features Engineering</a>

In [943]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [944]:
# Read data
df = pd.read_json('data/mini_sparkify_event_data.json', orient='records', lines=True)
df.head(5)

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771


## Data Wrangling

In [945]:

# drop anonymous
df = df[df['userId'] != '']

# convert userid type from string to numeric
df['userId'] = df['userId'].astype('int')

In [946]:
# custom function for lambda. convert raw numeric timestamp to datetime format.
def convert_ts(x):
    dt = datetime.utcfromtimestamp(int(x) / 1000)
    return dt

In [947]:
# Add event log datetime column. (human readable format) based on ts column
df['datetime'] = df.ts.apply(convert_ts)

# Add user registration datetime column. For later tenure calculation.
df['reg'] = df['registration'].apply(convert_ts)

In [948]:
# Custom lambda function. Convert useragent to device

def convert_ua(x):
    user_os = x.lower().split('/')[1]

    if 'windows' in user_os:
        x = 'Windows'
    elif 'macintosh' in user_os:
        x = 'MacOS'
    elif 'iphone' in user_os:
        x = 'iOS'
    elif 'ipad' in user_os:
        x = 'iOS'
    elif 'linux' in user_os:
        x = 'Linux'
    else:
        x = 'Others'

    return x


df['device'] = df['userAgent'].apply(convert_ua)

In [949]:
# Custom lambda function. Convert location to region

# https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States#/media/File:Census_Regions_and_Division_of_the_United_States.svg


def convert_location(x):
    west = ['wa', 'or', 'ca', 'az', 'nm', 'co', 'wy', 'mt', 'id', 'nv', 'ut', 'ak']
    midwest = ['nd', 'sd', 'ne', 'ks', 'mn', 'ia', 'mo', 'wi', 'il', 'in', 'mi', 'oh']
    northeast = ['pa', 'ny', 'nj', 'vt', 'ma', 'ct', 'nh', 'ri', 'me']
    south = ['tx', 'ok', 'ar', 'la', 'ms', 'ky', 'tn', 'al', 'ga', 'fl', 'wv', 'va', 'nc', 'sc', 'dc', 'md', 'de']

    location = x.lower().split(',')[1][1:3]

    if location in west:
        region = 'West'
    elif location in midwest:
        region = 'Midwest'
    elif location in northeast:
        region = 'Northeast'
    elif location in south:
        region = 'South'
    else:
        # print(location)
        region = 'Others'

    return region


df['region'] = df['location'].apply(convert_location)

In [950]:
# Check http status code percentage

df['status'].value_counts() / df['status'].value_counts().sum()

200    0.915745
307    0.083350
404    0.000906
Name: status, dtype: float64

## Measureing Churn

In [951]:
# Number of all distinct users

n_users = df['userId'].nunique()
n_users

225

In [952]:
# Number of 'paid' level users

n_paid_users = df[df['level'] == 'paid']['userId'].nunique()

In [953]:
# List of 'paid' users IDs

paid_user_ids = df[df['level'] == 'paid']['userId'].value_counts().index.to_list()

print(n_paid_users)
paid_user_ids

165


[39,
 92,
 140,
 300011,
 124,
 300021,
 300017,
 42,
 85,
 6,
 29,
 300015,
 9,
 100,
 91,
 54,
 126,
 118,
 74,
 51,
 98,
 136,
 65,
 4,
 81,
 15,
 132,
 142,
 41,
 138,
 88,
 300023,
 200004,
 127,
 101,
 300001,
 300018,
 128,
 200008,
 115,
 96,
 97,
 60,
 58,
 300002,
 62,
 20,
 25,
 120,
 113,
 104,
 131,
 200023,
 300009,
 300008,
 70,
 53,
 37,
 45,
 33,
 300025,
 82,
 38,
 14,
 152,
 114,
 145,
 69,
 95,
 61,
 200021,
 28,
 83,
 77,
 153,
 67,
 100013,
 103,
 17,
 52,
 35,
 46,
 300019,
 75,
 40,
 141,
 49,
 2,
 105,
 106,
 100008,
 36,
 155,
 66,
 10,
 109,
 300012,
 300016,
 111,
 100018,
 89,
 59,
 30,
 16,
 12,
 86,
 23,
 21,
 121,
 108,
 100004,
 200009,
 100016,
 56,
 100007,
 18,
 44,
 99,
 147,
 100023,
 13,
 300022,
 73,
 200025,
 200019,
 139,
 100025,
 300013,
 200017,
 100015,
 129,
 200011,
 200002,
 300014,
 200020,
 300005,
 100009,
 100014,
 300020,
 19,
 100012,
 3,
 93,
 71,
 11,
 100002,
 200024,
 200014,
 151,
 50,
 200003,
 300006,
 87,
 300004,
 300007,


In [954]:
# List of 'free' users IDs

free_user_ids = df[~df['userId'].isin(paid_user_ids)]['userId'].value_counts().index.to_list()

n_free_users = len(free_user_ids)
print(n_free_users)

free_user_ids

60


[100022,
 146,
 200006,
 148,
 200018,
 80,
 200022,
 117,
 100010,
 300010,
 200015,
 8,
 78,
 100021,
 200010,
 107,
 112,
 27,
 76,
 200016,
 47,
 43,
 149,
 110,
 150,
 119,
 5,
 100005,
 102,
 7,
 123,
 94,
 100001,
 200001,
 143,
 144,
 154,
 72,
 57,
 63,
 84,
 100019,
 200012,
 116,
 100003,
 100017,
 34,
 64,
 134,
 68,
 90,
 100006,
 133,
 22,
 100024,
 300003,
 100011,
 125,
 156,
 135]

In [955]:
# Spot Check: Number of 'authentication' values check

df['auth'].value_counts()

Logged In    278102
Cancelled        52
Name: auth, dtype: int64

In [956]:
# List of churn users

churn_user_ids = df[df['auth'] == 'Cancelled']['userId'].value_counts().index.to_list()

n_total_churn = len(churn_user_ids)

print(n_total_churn)

52


In [957]:
# Number of churn users in different subscription levels

df[df['auth'] == 'Cancelled'].groupby('level').size()

level
free    21
paid    31
dtype: int64

In [958]:
# Overall Total Churn Rate.
# Total number of churn users / total number of users

n_total_churn / n_users

0.2311111111111111

In [959]:
# Free users churn
# total number of churn users who are 'free-tier' / total number of free users

df[df['auth'] == 'Cancelled'].groupby('level').size()['free'] / n_free_users

0.35

In [960]:
# paid users churn
# total number of churn users who are  'paid-tier' / total number of paid users

df[df['auth'] == 'Cancelled'].groupby('level').size()['paid'] / n_paid_users

0.18787878787878787

In [961]:
# Spot Check: earliest date and latest event date in the dataset
print(np.min(df['datetime']))
print(np.max(df['datetime']))

# Spot Check: Date range in the dataset
print(np.max(df['datetime']) - np.min(df['datetime']))

# Spot Check: earliest date and latest user registration date in the dataset
print(np.min(df['reg']))
print(np.max(df['reg']))

2018-10-01 00:01:57
2018-12-03 01:11:16
63 days 01:09:19
2018-03-18 13:44:35
2018-11-26 15:49:14


<a id='features-engineering'></a>
## Features Engineering (Metrics Design)

In [962]:
df['region'].value_counts() / np.sum(df['region'].value_counts())

South        0.368605
West         0.260410
Northeast    0.255732
Midwest      0.115253
Name: region, dtype: float64

In [963]:
# Custom lambda function: compute tenure

def compute_tenure(r, d):
    # event datetime minus user reg datetime
    return (pd.Timestamp(d) - pd.Timestamp(r)).days


# expensive to calculate for each event date, might just calculate with every user's latest event date. (summary stats)
# ATTN: This tenure for every single event log. (may take times depend on dataset size)
df['tenure'] = df.apply(lambda x: compute_tenure(x.reg, x.datetime), axis=1)

In [964]:
# The plot below shows evidence that churn users having less tenure days.

avg_tenure_for_churn_users = np.mean(df[df.userId.isin(churn_user_ids)].groupby('userId')['tenure'].max())
avg_tenure_for_retain_users = np.mean(df[~df.userId.isin(churn_user_ids)].groupby('userId')['tenure'].max())

df_tenure_check = pd.DataFrame({
    'churn': ['Churn', 'Retain'],
    'tenure': [avg_tenure_for_churn_users, avg_tenure_for_retain_users]
})

df_tenure_check = df_tenure_check.set_index('churn')

df_tenure_check


,tenure
churn,
Churn,56.807692
Retain,86.144509


In [965]:

px.bar(df_tenure_check,
       x=df_tenure_check.index,
       y='tenure',
       title='average tenure days for churn, retain users'.title(),
       width=600,
       ).show()

In [966]:
# Check for category data: is UserAgent Device a matter to Churn?

# Churn user device perc
# NOTE that it is NOT necessary to say particular device users are more likely to churn. It is somehow be proportional to all user bases. No evidence.
df[df.userId.isin(churn_user_ids)].drop_duplicates('userId').device.value_counts() / np.sum(
    df[df.userId.isin(churn_user_ids)].drop_duplicates('userId').device.value_counts())

Windows    0.461538
MacOS      0.365385
Linux      0.096154
iOS        0.076923
Name: device, dtype: float64

In [967]:
# all users device perc

df.drop_duplicates('userId').device.value_counts() / np.sum(df.drop_duplicates('userId').device.value_counts())

Windows    0.493333
MacOS      0.382222
iOS        0.071111
Linux      0.053333
Name: device, dtype: float64

In [968]:
# Check for category data: is LOCATION REGION a matter to Churn?

# Churn user region perc
# NOTE that it is NOT necessary to say south users are more likely to churn. Might just be proportional to user bases. No evidence.
df_region_perc_churn = pd.DataFrame(
    df[df.userId.isin(churn_user_ids)].drop_duplicates('userId').region.value_counts() /
    np.sum(df[df.userId.isin(churn_user_ids)].drop_duplicates('userId').region.value_counts())
).reset_index()

df_region_perc_churn.columns = ['region', 'perc']
df_region_perc_churn = df_region_perc_churn.sort_values('region')

df_region_perc_all = pd.DataFrame(
    df.drop_duplicates('userId').region.value_counts() /
    np.sum(df.drop_duplicates('userId').region.value_counts())
).reset_index()

df_region_perc_all.columns = ['region', 'perc']
df_region_perc_all = df_region_perc_all.sort_values('region')

df_region_perc_check = pd.merge(df_region_perc_churn, df_region_perc_all, how="inner", on='region')
df_region_perc_check.columns = ['region', 'churn_base', 'all_base']

df_region_perc_check

,region,churn_base,all_base
0,Midwest,0.192308,0.164444
1,Northeast,0.153846,0.208889
2,South,0.384615,0.360000
3,West,0.269231,0.266667


In [969]:
# They are rather proportional. There is no typical evidence that region is factor of churn.

fig = px.bar(df_region_perc_check,
             x='region',
             y=['churn_base', 'all_base'],
             barmode='group',
             width=800,
             title="user percentage in region for churn and all user.".title(),
             labels={'region': 'Region', 'value': 'Percentage'}
             )
fig.show()

In [971]:
# Check for category data: is GENDER a matter to Churn?

# Churn user gender perc

df_gender_perc_churn = df[df.userId.isin(churn_user_ids)].drop_duplicates('userId').gender.value_counts() /
                       np.sum(df[df.userId.isin(churn_user_ids)].drop_duplicates('userId').gender.value_counts())

df_gender_perc_churn

M    0.615385
F    0.384615
Name: gender, dtype: float64

In [972]:
# All user gender perc

df_gender_perc_all = df.drop_duplicates('userId').gender.value_counts() /
                     np.sum(df.drop_duplicates('userId').gender.value_counts())

df_gender_perc_all

M    0.537778
F    0.462222
Name: gender, dtype: float64

In [973]:
df_gender_perc_check = pd.merge(df_gender_perc_churn, df_gender_perc_all, how='inner', on=df_gender_perc_churn.index)
df_gender_perc_check.columns = (['gender', 'churn_base', 'all_base'])
df_gender_perc_check

,gender,churn_base,all_base
0,M,0.615385,0.537778
1,F,0.384615,0.462222


In [974]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_gender_perc_check['gender'],
    y=df_gender_perc_check['churn_base'],
    marker_color='indianred',
    name='Churn Base',
    base=0
))

fig.add_trace(go.Bar(
    x=df_gender_perc_check['gender'],
    y=df_gender_perc_check['all_base'],
    hovertext=df_gender_perc_check['all_base'],
    marker_color='lightsalmon',
    name='All Base',
    base=-1
))

fig.update_layout(barmode='relative',
                  width=400,
                  height=500,
                  title="Relative Gender Percentage Compare",
                  )

fig.update_yaxes(visible=False)

fig.show()

⬆️ As we can see, within all users, genders are similar. However, for churn users, Male are relative more than Female. (62% vs 38%).
It might indicate Male are more likely to churn than Female.

In [975]:
# Another way to visualize the gender.

fig = go.Figure()
fig.add_trace(go.Bar(
    x=['Churn Base', 'All Base'],
    y=df_gender_perc_check.iloc[0].values[1:],
    marker_color='lightslategrey',
    name='Male',
))

fig.add_trace(go.Bar(
    x=['Churn Base', 'All Base'],
    y=df_gender_perc_check.iloc[1].values[1:],
    marker_color='hotpink',
    name='Female',
))

fig.update_layout(barmode='group',
                  width=400,
                  height=500,
                  title="Relative Gender Percentage Compare",
                  yaxis=dict(
                      title='Percentage',
                      titlefont_size=16,
                      tickfont_size=14),
                  xaxis=dict(
                      title="Comparison Base")
                  )

fig.show()

Therefore, for Categorical data (Nominal Data), I will select Gender. Exclude UserAgent, Location.

### Creating analytic dataset

In [977]:
# Drop some columns that won't select to be metrics.

df = df.drop(
    ['ts', 'method', 'status', 'location', 'userAgent', 'lastName', 'firstName', 'registration', 'device'], axis=1)

df.head(5)

KeyError: "['ts' 'method' 'status' 'location' 'userAgent' 'lastName' 'firstName'\n 'registration' 'device'] not found in axis"

In [978]:
# Transform to a schema. Summary dataset for each user. Having all user ids

df_analytic = df.groupby('userId')['gender'].max().reset_index()
df_analytic

,userId,gender
0,2,F
1,3,M
2,4,M
3,5,M
4,6,M
...,...,...
220,300021,F
221,300022,M
222,300023,F
223,300024,F


In [982]:
# remove event log for churn users that are after churn. otherwise, may create confusion for model
# in another words, user churn then continuous to use. need to find out the after churn (cancel) data, and trim them.

churn_ids_and_times = df[df.auth == 'Cancelled'][['userId', 'datetime']].to_numpy()

# Check if there are any event after user's cancel.
n_event_after_churn = 0

for i in churn_ids_and_times:
    uid = i[0]
    dt = i[1]
    idx = df[(df.userId == uid) & (df.datetime > dt)].index
    n_event_after_churn += len(idx)

n_event_after_churn

# No event after user cancel.

0

In [1171]:
# Get number of song play sessions (those contain song play event) for each user.

# Sessions that have song play event
df_n_session = pd.DataFrame(df.query("page == 'NextSong'").groupby(['userId', 'sessionId']).size()).reset_index().drop(
    [0], axis=1)

df_n_session = pd.DataFrame(df_n_session.groupby(['userId'])['sessionId'].count()).reset_index()

df_n_session.columns = ['userId', 'n_session']

# Outer Join to make sure retain all user ids
df_n_session = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_session,
    how='outer',
    on='userId'
)

df_n_session

,userId,n_session
0,2,7
1,3,4
2,4,22
3,5,5
4,6,22
...,...,...
220,300021,58
221,300022,11
222,300023,27
223,300024,1


In [1172]:
# Overview of distribution of the number of song play sessions

px.histogram(df_n_session,
             x="n_session",
             nbins=20,
             width=600,
             opacity=0.6,
             labels={'n_session': 'Number of Song Play Sessions'},
             title='Histogram of distribution of n_session',
             ).show()


In [1041]:
# Out of curiosity. on what day people tend to have more song playing session? (Monday to Sunday)

df_day_name = df.query("page == 'NextSong'").drop_duplicates(['userId', 'sessionId'])[['datetime']]

df_day_name['day_name'] = df_day_name['datetime'].apply(lambda x: x.day_name())

px.histogram(df_day_name,
             x="day_name",
             nbins=20,
             width=600,
             opacity=0.6,
             labels={'day_name': 'Day of Week'},
             title='Histogram of distribution of song play session of day of week',
             ).show()

# It is kind of surprising that I though for consumer product service like music,movie play, weekend expect more. However, the data plot
# reveals Sparkify user play more on the weekdays.

In [1175]:
# n_item
# total number of items in session for each user

df_n_item =\
    df.query("page == 'NextSong'").sort_values(by='userId').groupby(['userId', 'sessionId'])['itemInSession'].max().reset_index()\
        .groupby('userId')['itemInSession'].sum().reset_index()

df_n_item.columns = ['userId', 'n_item']

# Outer Join to make sure retain all user ids
df_n_item = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_item,
    how='outer',
    on='userId'
)

df_n_item

,userId,n_item
0,2,904
1,3,251
2,4,2456
3,5,215
4,6,3813
...,...,...
220,300021,4660
221,300022,436
222,300023,2199
223,300024,103


In [1176]:
# total_length

df_total_length = df.groupby(['userId'])['length'].sum().reset_index()

df_total_length.columns = ['userId', 'total_length']

# Outer Join to make sure retain all user ids
df_total_length = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_total_length,
    how='outer',
    on='userId'
)

df_total_length

,userId,total_length
0,2,188687.38342
1,3,54424.74544
2,4,506140.04138
3,5,39525.04698
4,6,787236.52359
...,...,...
220,300021,941922.73652
221,300022,91403.82518
222,300023,447755.12315
223,300024,22698.64417


In [1177]:
# n_songplay

df_n_songplay = df.query("page == 'NextSong'").groupby(['userId']).size().reset_index()

df_n_songplay.columns = ['userId', 'n_songplay']

# Outer Join to make sure retain all user ids
df_n_songplay = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_songplay,
    how='outer',
    on='userId'
)

df_n_songplay

,userId,n_songplay
0,2,755
1,3,214
2,4,2048
3,5,161
4,6,3159
...,...,...
220,300021,3816
221,300022,364
222,300023,1797
223,300024,88


In [1178]:
# n_like

df_n_like = df.query("page == 'Thumbs Up'").groupby(['userId']).size().reset_index()
df_n_like.columns = ['userId', 'n_like']

# Outer Join to make sure retain all user ids
df_n_like = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_like,
    how='outer',
    on='userId'
)

df_n_like

,userId,n_like
0,2,29.0
1,3,14.0
2,4,95.0
3,5,11.0
4,6,165.0
...,...,...
220,300021,336.0
221,300022,25.0
222,300023,169.0
223,300024,6.0


In [1180]:
# n_dislike

df_n_dislike = df.query("page == 'Thumbs Down'").groupby(['userId']).size().reset_index()
df_n_dislike.columns = ['userId', 'n_dislike']

# Outer Join to make sure retain all user ids
df_n_dislike = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_dislike,
    how='outer',
    on='userId'
)

df_n_dislike

,userId,n_dislike
0,2,6.0
1,3,3.0
2,4,26.0
3,5,NaN
4,6,31.0
...,...,...
220,300021,36.0
221,300022,5.0
222,300023,16.0
223,300024,NaN


In [1181]:
# n_addtoplaylist

df_n_addtoplaylist = df.query("page == 'Add to Playlist'").groupby(['userId']).size().reset_index()
df_n_addtoplaylist.columns = ['userId', 'n_addtoplaylist']

# Outer Join to make sure retain all user ids
df_n_addtoplaylist = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_addtoplaylist,
    how='outer',
    on='userId'
)

df_n_addtoplaylist

,userId,n_addtoplaylist
0,2,13.0
1,3,4.0
2,4,59.0
3,5,8.0
4,6,83.0
...,...,...
220,300021,107.0
221,300022,9.0
222,300023,63.0
223,300024,2.0


In [1182]:
# n_addfriend

df_n_addfriend = df.query("page == 'Add Friend'").groupby(['userId']).size().reset_index()
df_n_addfriend.columns = ['userId', 'n_addfriend']

# Outer Join to make sure retain all user ids
df_n_addfriend = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_addfriend,
    how='outer',
    on='userId'
)

df_n_addfriend

,userId,n_addfriend
0,2,20.0
1,3,1.0
2,4,46.0
3,5,3.0
4,6,41.0
...,...,...
220,300021,89.0
221,300022,11.0
222,300023,31.0
223,300024,2.0


In [1183]:
# n_adview

df_n_adview = df.query("page == 'Roll Advert'").groupby(['userId']).size().reset_index()
df_n_adview.columns = ['userId', 'n_adview']

# Outer Join to make sure retain all user ids
df_n_adview = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_adview,
    how='outer',
    on='userId'
)

df_n_adview

,userId,n_adview
0,2,NaN
1,3,1.0
2,4,4.0
3,5,11.0
4,6,9.0
...,...,...
220,300021,5.0
221,300022,NaN
222,300023,2.0
223,300024,NaN


In [1184]:
# n_viewdowngrade

df_n_viewdowngrade = df.query("page == 'Downgrade'").groupby(['userId']).size().reset_index()
df_n_viewdowngrade.columns = ['userId', 'n_viewdowngrade']

# Outer Join to make sure retain all user ids
df_n_viewdowngrade = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_viewdowngrade,
    how='outer',
    on='userId'
)

df_n_viewdowngrade

,userId,n_viewdowngrade
0,2,13.0
1,3,2.0
2,4,21.0
3,5,NaN
4,6,41.0
...,...,...
220,300021,17.0
221,300022,1.0
222,300023,12.0
223,300024,NaN


In [1185]:
# n_artist

df_n_artist = df[~df.artist.isnull()].groupby(['userId', 'artist']).size().reset_index()\
    .groupby(['userId'])['artist'].count().reset_index()

df_n_artist.columns = ['userId', 'n_artist']

# Outer Join to make sure retain all user ids
df_n_artist = pd.merge(
    df_analytic.drop(['gender'], axis=1),
    df_n_artist,
    how='outer',
    on='userId'
)

df_n_artist

,userId,n_artist
0,2,587
1,3,197
2,4,1342
3,5,154
4,6,1868
...,...,...
220,300021,2167
221,300022,328
222,300023,1235
223,300024,85


In [1186]:
# ratio: length_per_session

df_ratio_length_per_session = pd.merge(
    # Join data. Retain all possible users ids with outer join.
    # Outer Join guarantee to have all keys on left table.
    pd.merge(df_analytic.drop(['gender'], axis=1), df_total_length, how='outer', on='userId'),
    df_n_session, how='outer', on='userId'
)

df_ratio_length_per_session['len_per_sess'] = df_ratio_length_per_session['total_length'] / df_ratio_length_per_session['n_session']

df_ratio_length_per_session


,userId,total_length,n_session,len_per_sess
0,2,188687.38342,7,26955.340489
1,3,54424.74544,4,13606.186360
2,4,506140.04138,22,23006.365517
3,5,39525.04698,5,7905.009396
4,6,787236.52359,22,35783.478345
...,...,...,...,...
220,300021,941922.73652,58,16240.047181
221,300022,91403.82518,11,8309.438653
222,300023,447755.12315,27,16583.523080
223,300024,22698.64417,1,22698.644170


In [1187]:
# ratio: addtoplaylist_per_songplay

df_ratio_addtoplaylist_per_songplay = pd.merge(
    # Join data. Retain all possible users ids with outer join.
    # Outer Join guarantee to have all keys on left table.
    pd.merge(df_analytic.drop(['gender'], axis=1), df_n_addtoplaylist, how='outer', on='userId'),
    df_n_songplay, how='outer', on='userId'
)

df_ratio_addtoplaylist_per_songplay['addtoplaylist_per_play'] = \
    df_ratio_addtoplaylist_per_songplay.n_addtoplaylist / df_ratio_addtoplaylist_per_songplay.n_songplay

df_ratio_addtoplaylist_per_songplay

,userId,n_addtoplaylist,n_songplay,addtoplaylist_per_play
0,2,13.0,755,0.017219
1,3,4.0,214,0.018692
2,4,59.0,2048,0.028809
3,5,8.0,161,0.049689
4,6,83.0,3159,0.026274
...,...,...,...,...
220,300021,107.0,3816,0.028040
221,300022,9.0,364,0.024725
222,300023,63.0,1797,0.035058
223,300024,2.0,88,0.022727


In [1188]:
# ratio: like_per_songplay

df_ratio_like_per_songplay = pd.merge(
    # Join data. Retain all possible users ids with outer join.
    # Outer Join guarantee to have all keys on left table.
    pd.merge(df_analytic.drop(['gender'], axis=1), df_n_like, how='outer', on='userId'),
    df_n_songplay, how='outer', on='userId'
)

df_ratio_like_per_songplay['like_per_songplay'] = \
    df_ratio_like_per_songplay.n_like / df_ratio_like_per_songplay.n_songplay

df_ratio_like_per_songplay

,userId,n_like,n_songplay,like_per_songplay
0,2,29.0,755,0.038411
1,3,14.0,214,0.065421
2,4,95.0,2048,0.046387
3,5,11.0,161,0.068323
4,6,165.0,3159,0.052232
...,...,...,...,...
220,300021,336.0,3816,0.088050
221,300022,25.0,364,0.068681
222,300023,169.0,1797,0.094046
223,300024,6.0,88,0.068182


In [1189]:
# ratio: dislike_per_songplay

df_ratio_dislike_per_songplay = pd.merge(
    # Join data. Retain all possible users ids with outer join.
    # Outer Join guarantee to have all keys on left table.
    pd.merge(df_analytic.drop(['gender'], axis=1), df_n_dislike, how='outer', on='userId'),
    df_n_songplay, how='outer', on='userId'
)

df_ratio_dislike_per_songplay['dislike_per_songplay'] = \
    df_ratio_dislike_per_songplay.n_dislike / df_ratio_dislike_per_songplay.n_songplay

df_ratio_dislike_per_songplay

,userId,n_dislike,n_songplay,dislike_per_songplay
0,2,6.0,755,0.007947
1,3,3.0,214,0.014019
2,4,26.0,2048,0.012695
3,5,NaN,161,NaN
4,6,31.0,3159,0.009813
...,...,...,...,...
220,300021,36.0,3816,0.009434
221,300022,5.0,364,0.013736
222,300023,16.0,1797,0.008904
223,300024,NaN,88,NaN


In [1107]:
# Change measurement variables
# define datetime range for cycle comparison

interval = pd.Timedelta('28 days')

datetime_latest_to = pd.Timestamp('2018-12-03')
datetime_latest_from = pd.Timestamp('2018-12-03') - interval

datetime_previous_to = datetime_latest_from
datetime_previous_from = datetime_previous_to - interval

print(f"Latest Cycle vs Previous Cycle")
print(f"Each measurement cycle is 28 days duration.")
print(f"Latest Cycle: [{datetime_latest_from} - {datetime_latest_to})")
print(f"Previous Cycle: [{datetime_previous_from} - {datetime_previous_to})")

Latest Cycle vs Previous Cycle
Each measurement cycle is 28 days duration.
Latest Cycle: [2018-11-05 00:00:00 - 2018-12-03 00:00:00)
Previous Cycle: [2018-10-08 00:00:00 - 2018-11-05 00:00:00)


In [1192]:
# Change: change_perc_n_songplay

# df[(df.page == 'NextSong') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index()
# df[(df.page == 'NextSong') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index()


df_change_perc_n_songplay = pd.merge(
    left=df_analytic.drop(['gender'], axis=1),
    right=df[(df.page == 'NextSong') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)

df_change_perc_n_songplay = pd.merge(
    left=df_change_perc_n_songplay,
    right=df[(df.page == 'NextSong') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)

df_change_perc_n_songplay.columns = ['userId', 'previous', 'latest']



df_change_perc_n_songplay['change_perc_n_songplay'] = \
    (df_change_perc_n_songplay.latest - df_change_perc_n_songplay.previous) / df_change_perc_n_songplay.previous

# fill result of NaN to zero.
df_change_perc_n_songplay['change_perc_n_songplay'] = df_change_perc_n_songplay['change_perc_n_songplay'].fillna(0)

df_change_perc_n_songplay

,userId,previous,latest,change_perc_n_songplay
0,2,508.0,247.0,-0.513780
1,3,192.0,NaN,0.000000
2,4,1004.0,1025.0,0.020916
3,5,107.0,54.0,-0.495327
4,6,1818.0,1219.0,-0.329483
...,...,...,...,...
220,300021,2141.0,1330.0,-0.378795
221,300022,199.0,165.0,-0.170854
222,300023,1019.0,672.0,-0.340530
223,300024,88.0,NaN,0.000000


In [1193]:
# Change: change_perc_n_addtoplaylist

# df[(df.page == 'Add to Playlist') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index()
# df[(df.page == 'Add to Playlist') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index()


df_change_perc_n_addtoplaylist = pd.merge(
    left=df_analytic.drop(['gender'], axis=1),
    right=df[(df.page == 'Add to Playlist') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)

df_change_perc_n_addtoplaylist = pd.merge(
    left=df_change_perc_n_addtoplaylist,
    right=df[(df.page == 'Add to Playlist') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)


df_change_perc_n_addtoplaylist.columns = ['userId', 'previous', 'latest']


df_change_perc_n_addtoplaylist['change_perc_n_addtoplaylist'] = \
    (df_change_perc_n_addtoplaylist.latest - df_change_perc_n_addtoplaylist.previous) / df_change_perc_n_addtoplaylist.previous

# fill result of NaN to zero.
df_change_perc_n_addtoplaylist['change_perc_n_addtoplaylist'] = df_change_perc_n_addtoplaylist['change_perc_n_addtoplaylist'].fillna(0)

df_change_perc_n_addtoplaylist

,userId,previous,latest,change_perc_n_addtoplaylist
0,2,6.0,7.0,0.166667
1,3,4.0,NaN,0.000000
2,4,24.0,35.0,0.458333
3,5,6.0,2.0,-0.666667
4,6,45.0,32.0,-0.288889
...,...,...,...,...
220,300021,59.0,39.0,-0.338983
221,300022,4.0,5.0,0.250000
222,300023,37.0,22.0,-0.405405
223,300024,2.0,NaN,0.000000


In [1194]:
# Change: change_perc_n_like

# df[(df.page == 'Thumbs Up') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index()
# df[(df.page == 'Thumbs Up') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index()


df_change_perc_n_like = pd.merge(
    left=df_analytic.drop(['gender'], axis=1),
    right=df[(df.page == 'Thumbs Up') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)

df_change_perc_n_like = pd.merge(
    left=df_change_perc_n_like,
    right=df[(df.page == 'Thumbs Up') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)

df_change_perc_n_like.columns = ['userId', 'previous', 'latest']



df_change_perc_n_like['change_perc_n_like'] = \
    (df_change_perc_n_like.latest - df_change_perc_n_like.previous) / df_change_perc_n_like.previous

# fill result of NaN to zero.
df_change_perc_n_like['change_perc_n_like'] = df_change_perc_n_like['change_perc_n_like'].fillna(0)

df_change_perc_n_like

,userId,previous,latest,change_perc_n_like
0,2,21.0,8.0,-0.619048
1,3,13.0,NaN,0.000000
2,4,42.0,53.0,0.261905
3,5,7.0,4.0,-0.428571
4,6,86.0,69.0,-0.197674
...,...,...,...,...
220,300021,196.0,111.0,-0.433673
221,300022,15.0,10.0,-0.333333
222,300023,87.0,64.0,-0.264368
223,300024,6.0,NaN,0.000000


In [1195]:
# Change: change_perc_n_dislike

# df[(df.page == 'Thumbs Down') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index()
# df[(df.page == 'Thumbs Down') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index()


df_change_perc_n_dislike = pd.merge(
    left=df_analytic.drop(['gender'], axis=1),
    right=df[(df.page == 'Thumbs Down') & (df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)

df_change_perc_n_dislike = pd.merge(
    left=df_change_perc_n_dislike,
    right=df[(df.page == 'Thumbs Down') & (df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId']).size().reset_index(),
    how='left',
    on='userId'
)

df_change_perc_n_dislike.columns = ['userId', 'previous', 'latest']



df_change_perc_n_dislike['change_perc_n_dislike'] = \
    (df_change_perc_n_dislike.latest - df_change_perc_n_dislike.previous) / df_change_perc_n_dislike.previous

# fill result of NaN to zero.
df_change_perc_n_dislike['change_perc_n_dislike'] = df_change_perc_n_dislike['change_perc_n_dislike'].fillna(0)

df_change_perc_n_dislike

,userId,previous,latest,change_perc_n_dislike
0,2,3.0,3.0,0.000000
1,3,3.0,NaN,0.000000
2,4,15.0,11.0,-0.266667
3,5,NaN,NaN,0.000000
4,6,19.0,11.0,-0.421053
...,...,...,...,...
220,300021,20.0,12.0,-0.400000
221,300022,3.0,2.0,-0.333333
222,300023,9.0,7.0,-0.222222
223,300024,NaN,NaN,0.000000


In [1196]:
# Change: change_perc_total_length

# df[(df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId'])['length'].sum()
# df[(df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId'])['length'].sum()

df_change_perc_total_length = pd.merge(
    left=df_analytic.drop(['gender'], axis=1),
    right=df[(df.datetime >= datetime_previous_from) & (df.datetime < datetime_previous_to)].groupby(['userId'])['length'].sum(),
    how='left',
    on='userId'
)

df_change_perc_total_length = pd.merge(
    left=df_change_perc_total_length,
    right=df[(df.datetime >= datetime_latest_from) & (df.datetime < datetime_latest_to)].groupby(['userId'])['length'].sum(),
    how='left',
    on='userId'
)

df_change_perc_total_length.columns = ['userId', 'previous', 'latest']


df_change_perc_total_length['change_perc_total_length'] = \
    (df_change_perc_total_length['latest'] - df_change_perc_total_length['previous']) / df_change_perc_total_length['previous']

# fill result of NaN to zero. default to zero.
df_change_perc_total_length['change_perc_total_length'] = df_change_perc_total_length['change_perc_total_length'].fillna(0)

df_change_perc_total_length

,userId,previous,latest,change_perc_total_length
0,2,126562.53867,62124.84475,-0.509137
1,3,49060.74560,NaN,0.000000
2,4,247629.18078,252846.11166,0.021068
3,5,25923.71308,13601.33390,-0.475332
4,6,451570.50628,303683.96646,-0.327494
...,...,...,...,...
220,300021,531830.72212,325751.24670,-0.387491
221,300022,49419.09016,41984.73502,-0.150435
222,300023,251661.40886,167681.94194,-0.333700
223,300024,22698.64417,NaN,0.000000


In [1197]:
pd.concat([df_n_like, df_n_songplay], axis=1)

,userId,n_like,userId,n_songplay
0,2,29.0,2,755
1,3,14.0,3,214
2,4,95.0,4,2048
3,5,11.0,5,161
4,6,165.0,6,3159
...,...,...,...,...
220,300021,336.0,300021,3816
221,300022,25.0,300022,364
222,300023,169.0,300023,1797
223,300024,6.0,300024,88


In [1169]:
df_n_songplay

,userId,n_songplay
0,2,755
1,3,214
2,4,2048
3,5,161
4,6,3159
...,...,...
220,300021,3816
221,300022,364
222,300023,1797
223,300024,88
